<a href="https://colab.research.google.com/github/adityashah841/IPD/blob/main/KDTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import random
import math

class KDNode:
    def __init__(self, point, split_dim, data=None, left=None, right=None):
        self.point = point
        self.split_dim = split_dim
        self.data = data
        self.left = left
        self.right = right

class KDTree:
    def __init__(self, data=None):
        self.root = None
        if data:
            self.build(data)
    
    def build(self, data):
        def recursive_build(points, split_dim):
            if not points:
                return None
            
            points.sort(key=lambda x: x[0][split_dim])
            mid = len(points) // 2
            node = KDNode(points[mid][0], split_dim, points[mid][1])
            
            node.left = recursive_build(points[:mid], (split_dim + 1) % len(points[0][0]))
            node.right = recursive_build(points[mid+1:], (split_dim + 1) % len(points[0][0]))
            
            return node
        
        self.root = recursive_build(data, 0)
    
    def find_closest_node(self, point):
        closest_node = None
        min_dist = math.inf
        
        def traverse(node):
            nonlocal closest_node, min_dist
            
            if node is None:
                return
            
            dist = distance(point, node.point)
            
            if dist < min_dist:
                closest_node = node
                min_dist = dist
            
            if point[node.split_dim] < node.point[node.split_dim]:
                traverse(node.left)
                if (point[node.split_dim] - min_dist) <= node.point[node.split_dim]:
                    traverse(node.right)
            else:
                traverse(node.right)
                if (point[node.split_dim] + min_dist) >= node.point[node.split_dim]:
                    traverse(node.left)
        
        traverse(self.root)
        return closest_node
    
def distance(p1, p2):
    return math.sqrt(sum([(p1[i]-p2[i])**2 for i in range(len(p1))]))


if __name__ == "__main__":
    data = [((random.uniform(0, 10), random.uniform(0, 10)), random.randint(0, 9)) for i in range(10)]
    
    tree = KDTree(data)
    
    query_point = (5, 5)
    closest_node = tree.find_closest_node(query_point)
    print("Query point:", query_point)
    print("Closest point:", closest_node.point)
    print("Data associated with closest point:", closest_node.data)


Query point: (5, 5)
Closest point: (2.966830635592701, 6.654029452423585)
Data associated with closest point: 5


In [6]:
import pandas as pd

In [27]:
df = pd.read_csv('/content/us_hospital_locations.csv')

In [28]:
df.columns

Index(['X', 'Y', 'FID', 'ID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP',
       'ZIP4', 'TELEPHONE', 'TYPE', 'STATUS', 'POPULATION', 'COUNTY',
       'COUNTYFIPS', 'COUNTRY', 'LATITUDE', 'LONGITUDE', 'NAICS_CODE',
       'NAICS_DESC', 'SOURCE', 'SOURCEDATE', 'VAL_METHOD', 'VAL_DATE',
       'WEBSITE', 'STATE_ID', 'ALT_NAME', 'ST_FIPS', 'OWNER', 'TTL_STAFF',
       'BEDS', 'TRAUMA', 'HELIPAD'],
      dtype='object')

In [29]:
df2 = df.drop(['X', 'Y', 'FID', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'ZIP4', 'TYPE', 'STATUS',  'POPULATION', 'COUNTY',
       'COUNTYFIPS', 'COUNTRY', 'NAICS_CODE',
       'NAICS_DESC', 'SOURCE', 'SOURCEDATE', 'VAL_METHOD', 'VAL_DATE',
       'WEBSITE', 'STATE_ID', 'ALT_NAME', 'ST_FIPS', 'OWNER', 'TTL_STAFF',
       'BEDS', 'TRAUMA', 'HELIPAD'], axis = 1)

In [30]:
df2.columns

Index(['ID', 'NAME', 'TELEPHONE', 'LATITUDE', 'LONGITUDE'], dtype='object')

In [31]:
df2.head()

,ID,NAME,TELEPHONE,LATITUDE,LONGITUDE
0,5793230,CENTRAL VALLEY GENERAL HOSPITAL,NOT AVAILABLE,36.336159,-119.645667
1,53391362,LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CA...,NOT AVAILABLE,34.154939,-118.815736
2,11190023,EAST LOS ANGELES DOCTORS HOSPITAL,NOT AVAILABLE,34.023647,-118.184165
3,17090028,SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,(323) 462-2271,34.096391,-118.325235
4,23691706,KINDRED HOSPITAL BALDWIN PARK,NOT AVAILABLE,34.063039,-117.967438


In [32]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7596 entries, 0 to 7595
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         7596 non-null   int64  
 1   NAME       7596 non-null   object 
 2   TELEPHONE  7596 non-null   object 
 3   LATITUDE   7596 non-null   float64
 4   LONGITUDE  7596 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 296.8+ KB


In [33]:
df2 = df2[df2['TELEPHONE']!='NOT AVAILABLE']

In [34]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6554 entries, 3 to 7595
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         6554 non-null   int64  
 1   NAME       6554 non-null   object 
 2   TELEPHONE  6554 non-null   object 
 3   LATITUDE   6554 non-null   float64
 4   LONGITUDE  6554 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 307.2+ KB


In [35]:
df2.head()

,ID,NAME,TELEPHONE,LATITUDE,LONGITUDE
3,17090028,SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,(323) 462-2271,34.096391,-118.325235
12,12678414,CORPUS CHRISTI SPECIALTY HOSPITAL,(361) 888-4323,27.778510,-97.396208
19,5870563,DAUTERIVE HOSPITAL,(337) 365-7311,30.006907,-91.794524
20,13177954,CROCKETT MEDICAL CENTER,(936( 546-3891,31.323604,-95.438899
21,4671360,CENTRAL LOUISIANA STATE HOSPITAL,(318) 484-6200,31.327379,-92.440135


In [36]:
df2['DATA'] = df2['ID'].astype(str) + ', ' + df2['NAME'] + ', ' + df2['TELEPHONE']

In [37]:
df2.head()

,ID,NAME,TELEPHONE,LATITUDE,LONGITUDE,DATA
3,17090028,SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD,(323) 462-2271,34.096391,-118.325235,"17090028, SOUTHERN CALIFORNIA HOSPITAL AT HOLL..."
12,12678414,CORPUS CHRISTI SPECIALTY HOSPITAL,(361) 888-4323,27.778510,-97.396208,"12678414, CORPUS CHRISTI SPECIALTY HOSPITAL, (..."
19,5870563,DAUTERIVE HOSPITAL,(337) 365-7311,30.006907,-91.794524,"5870563, DAUTERIVE HOSPITAL, (337) 365-7311"
20,13177954,CROCKETT MEDICAL CENTER,(936( 546-3891,31.323604,-95.438899,"13177954, CROCKETT MEDICAL CENTER, (936( 546-3891"
21,4671360,CENTRAL LOUISIANA STATE HOSPITAL,(318) 484-6200,31.327379,-92.440135,"4671360, CENTRAL LOUISIANA STATE HOSPITAL, (31..."


In [39]:
df2.drop(['ID', 'NAME', 'TELEPHONE'], axis=1, inplace=True)

In [40]:
df2.head()

,LATITUDE,LONGITUDE,DATA
3,34.096391,-118.325235,"17090028, SOUTHERN CALIFORNIA HOSPITAL AT HOLL..."
12,27.778510,-97.396208,"12678414, CORPUS CHRISTI SPECIALTY HOSPITAL, (..."
19,30.006907,-91.794524,"5870563, DAUTERIVE HOSPITAL, (337) 365-7311"
20,31.323604,-95.438899,"13177954, CROCKETT MEDICAL CENTER, (936( 546-3891"
21,31.327379,-92.440135,"4671360, CENTRAL LOUISIANA STATE HOSPITAL, (31..."


In [41]:
len(df2)

6554

In [51]:
df2.iloc(0)[1]['LATITUDE']

27.778510168

In [55]:
df2.iloc(0)[0]['DATA']

'17090028, SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOOD, (323) 462-2271'

In [56]:
data = [((df2.iloc(0)[i]['LATITUDE'], df2.iloc(0)[i]['LONGITUDE']), df2.iloc(0)[i]['DATA']) for i in range(len(df2))]

In [57]:
tree = KDTree(data)

In [58]:
import pickle

with open('tree.pickle', 'wb') as f:
    pickle.dump(tree, f)

In [59]:
with open('tree.pickle', 'rb') as f:
    loaded_tree = pickle.load(f)

In [61]:
loaded_tree.find_closest_node((112,123)).point

(15.211634437, 145.724472394)